# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [1]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 3.1 MB/s eta 0:

In [2]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register
* returns: `None`

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [3]:
def oracle010(qq):
    # YOUR CODE HERE
    yield cirq.X(qq[0])
    yield cirq.X(qq[2])
    yield cirq.Z(qq[1]).controlled_by(qq[0], qq[2])
    yield cirq.X(qq[0])
    yield cirq.X(qq[2])


In [4]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────@───────
          │
2: ───X───@───X───

In [5]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [6]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator()
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [10]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'010': 96, '101': 1, '110': 1, '000': 1, '100': 1})


96

In [8]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Graph $G$ has 5 vertices and 6 edges: (0,3), (0,4), (1,3), (1,4), (2,3), (2,4).

Write an oracle for the graph $G$ to check whether it admits a valid 2-coloring.

The function `oracle2` has

* input: `qq`, a 12-qubit register
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.

In [11]:
def oracle2(qq):
    # YOUR CODE HERE
    def edge_check(a, b, c):
        yield CX(qq[a], qq[c])
        yield CX(qq[b], qq[c])


    yield edge_check(0, 3, 5)
    yield edge_check(0, 4, 6)
    yield edge_check(1, 3, 7)
    yield edge_check(1, 4, 8)
    yield edge_check(2, 3, 9)
    yield edge_check(2, 4, 10)

    yield X(qq[11]).controlled_by(*qq[5:11])

In [12]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))

In [13]:
def grover2(trials_number):
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [15]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'00011': 443,
         '11100': 459,
         '00110': 6,
         '01010': 4,
         '10100': 4,
         '00001': 5,
         '11001': 6,
         '01000': 7,
         '10001': 3,
         '10010': 2,
         '01101': 3,
         '01100': 1,
         '01001': 2,
         '11110': 4,
         '11011': 3,
         '00000': 6,
         '01110': 3,
         '10011': 7,
         '01111': 4,
         '11101': 2,
         '11111': 2,
         '10101': 3,
         '01011': 3,
         '10110': 2,
         '11010': 4,
         '00101': 2,
         '00010': 5,
         '00111': 1,
         '00100': 2,
         '11000': 1,
         '10111': 1})

In [16]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Graph $G$ has 4 vertices and 5 edges: (0,1), (0,2), (0,3), (1,2), (1,3)

Write an oracle for the graph $G$ to check whether there exists a coloring with at least 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits
- 0-3 for the vertices,
- 4-8 for the edges,
- 9-11 for the addition (remember we need three qubits here for addition unlike the last question), and
- 12 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.

In [36]:
def oracle3(qq):
    # YOUR CODE HERE



        yield cirq.CX(qq[0], qq[4])
        yield cirq.CX(qq[1], qq[4])

        yield cirq.CX(qq[0], qq[5])
        yield cirq.CX(qq[2], qq[5])

        yield cirq.CX(qq[0], qq[6])
        yield cirq.CX(qq[3], qq[6])

        yield cirq.CX(qq[1], qq[7])
        yield cirq.CX(qq[2], qq[7])

        yield cirq.CX(qq[1], qq[8])
        yield cirq.CX(qq[3], qq[8])

        yield cirq.CX(qq[4], qq[9])





    # add qubits 5-7
        for j in range(5,8):
            yield cirq.CCX(qq[j], qq[9], qq[10])
            yield cirq.CX(qq[j], qq[9])

    # add qubits 8
        yield cirq.X(qq[11]).controlled_by(qq[8], qq[9], qq[10])
        yield cirq.CCX(qq[8], qq[9], qq[10])
        yield cirq.CX(qq[8], qq[9])


    # Check if the num is == to 4
        yield cirq.X(qq[9])
        yield cirq.X(qq[10])
        yield cirq.X(qq[12]).controlled_by(*(qq[8:11]))

In [37]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))

In [38]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX

def grover3(trials_number):
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [44]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'1100': 473,
         '0011': 470,
         '1111': 7,
         '1000': 3,
         '1010': 4,
         '0110': 10,
         '1011': 4,
         '1001': 4,
         '0010': 3,
         '0100': 5,
         '0111': 4,
         '0101': 4,
         '1110': 4,
         '1101': 1,
         '0001': 2,
         '0000': 2})

In [42]:
# hidden tests in this cell will be used for grading.